In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

# Function to scrape product details from a product page
def scrape_product_details(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extracting product details
    product_details = {}
    product_details['Title'] = soup.h1.text.strip()
    product_details['UPC'] = soup.find('th', string='UPC').find_next('td').get_text(strip=True)
    product_details['Product_Type'] = soup.find('th', string='Product Type').find_next('td').get_text(strip=True)
    product_details['Price_excl_tax'] = soup.find('th', string='Price (excl. tax)').find_next('td').get_text(strip=True)
    product_details['Price_incl_tax'] = soup.find('th', string='Price (incl. tax)').find_next('td').get_text(strip=True)
    product_details['Tax'] = soup.find('th', string='Tax').find_next('td').get_text(strip=True)
    product_details['Availability'] = soup.find('th', string='Availability').find_next('td').get_text(strip=True)
    product_details['Description'] = soup.find('meta', attrs={'name': 'description'})['content']
    product_details['Reviews'] = soup.find('th', string='Number of reviews').find_next('td').get_text(strip=True)

    return product_details

# Function to scrape product data from a page
def scrape_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extracting product information
    products = []
    for product in soup.find_all('h3'):
        product_url = urljoin(url, product.a['href'])
        product_data = scrape_product_details(product_url)
        products.append(product_data)

    return products

# URL of the website
base_url = 'http://books.toscrape.com/catalogue/page-{}.html'

# Scrape data from the first 10 pages
all_products = []
for page_number in range(1, 11):
    page_url = base_url.format(page_number)
    all_products.extend(scrape_page(page_url))

# Create a DataFrame
df = pd.DataFrame(all_products)

# Display the DataFrame
print(df)


                                                 Title               UPC  \
0                                 A Light in the Attic  a897fe39b1053632   
1                                   Tipping the Velvet  90fa61229261140a   
2                                           Soumission  6957f44c3847a760   
3                                        Sharp Objects  e00eb4fd7b871a48   
4                Sapiens: A Brief History of Humankind  4165285e1663650f   
..                                                 ...               ...   
195                                  Eureka Trivia 6.0  29c0025455f8c585   
196  Drive: The Surprising Truth About What Motivat...  6ffb36aaeff1c81e   
197            Done Rubbed Out (Reightman & Bailey #1)  f6d967cdadc6fbd9   
198                  Doing It Over (Most Likely To #1)  3a11bb962ff45b78   
199  Deliciously Ella Every Day: Quick and Easy Rec...  46c1530d7546ea6d   

    Product_Type Price_excl_tax Price_incl_tax     Tax  \
0          Books        Â£51.

In [2]:
df

,Title,UPC,Product_Type,Price_excl_tax,Price_incl_tax,Tax,Availability,Description,Reviews
0,A Light in the Attic,a897fe39b1053632,Books,Â£51.77,Â£51.77,Â£0.00,In stock (22 available),\n It's hard to imagine a world without A L...,0
1,Tipping the Velvet,90fa61229261140a,Books,Â£53.74,Â£53.74,Â£0.00,In stock (20 available),"\n ""Erotic and absorbing...Written with sta...",0
2,Soumission,6957f44c3847a760,Books,Â£50.10,Â£50.10,Â£0.00,In stock (20 available),\n Dans une France assez proche de la nÃ´tr...,0
3,Sharp Objects,e00eb4fd7b871a48,Books,Â£47.82,Â£47.82,Â£0.00,In stock (20 available),"\n WICKED above her hipbone, GIRL across he...",0
4,Sapiens: A Brief History of Humankind,4165285e1663650f,Books,Â£54.23,Â£54.23,Â£0.00,In stock (20 available),\n From a renowned historian comes a ground...,0
...,...,...,...,...,...,...,...,...,...
195,Eureka Trivia 6.0,29c0025455f8c585,Books,Â£54.59,Â£54.59,Â£0.00,In stock (15 available),\n Eureka Trivia 6.0 is a great trivia book...,0
196,Drive: The Surprising Truth About What Motivat...,6ffb36aaeff1c81e,Books,Â£34.95,Â£34.95,Â£0.00,In stock (15 available),\n Forget everything you thought you knew a...,0
197,Done Rubbed Out (Reightman & Bailey #1),f6d967cdadc6fbd9,Books,Â£37.72,Â£37.72,Â£0.00,In stock (15 available),\n In this first book of the Reightman & Ba...,0
198,Doing It Over (Most Likely To #1),3a11bb962ff45b78,Books,Â£35.61,Â£35.61,Â£0.00,In stock (15 available),"\n Voted Most Likely to Succeed, Melanie Ba...",0


In [4]:
# Define normalize_text function
def normalize_text(text):
    return ''.join(c if c.isalnum() else '_' for c in text.lower())

columns_to_normalize = ['Title', 'UPC', 'Product_Type', 'Price_excl_tax', 'Price_incl_tax', 'Tax', 'Availability', 'Description']

# Normalize text in each column
for column in columns_to_normalize:
    df[column] = df[column].apply(normalize_text)


df


,Title,UPC,Product_Type,Price_excl_tax,Price_incl_tax,Tax,Availability,Description,Reviews
0,a_light_in_the_attic,a897fe39b1053632,books,â_51_77,â_51_77,â_0_00,in_stock__22_available_,_____it_s_hard_to_imagine_a_world_without_a_li...,0
1,tipping_the_velvet,90fa61229261140a,books,â_53_74,â_53_74,â_0_00,in_stock__20_available_,______erotic_and_absorbing___written_with_star...,0
2,soumission,6957f44c3847a760,books,â_50_10,â_50_10,â_0_00,in_stock__20_available_,_____dans_une_france_assez_proche_de_la_nã_tre...,0
3,sharp_objects,e00eb4fd7b871a48,books,â_47_82,â_47_82,â_0_00,in_stock__20_available_,_____wicked_above_her_hipbone__girl_across_her...,0
4,sapiens__a_brief_history_of_humankind,4165285e1663650f,books,â_54_23,â_54_23,â_0_00,in_stock__20_available_,_____from_a_renowned_historian_comes_a_groundb...,0
...,...,...,...,...,...,...,...,...,...
195,eureka_trivia_6_0,29c0025455f8c585,books,â_54_59,â_54_59,â_0_00,in_stock__15_available_,_____eureka_trivia_6_0_is_a_great_trivia_book_...,0
196,drive__the_surprising_truth_about_what_motivat...,6ffb36aaeff1c81e,books,â_34_95,â_34_95,â_0_00,in_stock__15_available_,_____forget_everything_you_thought_you_knew_ab...,0
197,done_rubbed_out__reightman___bailey__1_,f6d967cdadc6fbd9,books,â_37_72,â_37_72,â_0_00,in_stock__15_available_,_____in_this_first_book_of_the_reightman___bai...,0
198,doing_it_over__most_likely_to__1_,3a11bb962ff45b78,books,â_35_61,â_35_61,â_0_00,in_stock__15_available_,_____voted_most_likely_to_succeed__melanie_bar...,0


In [5]:
# Make 'Title' column values uppercase
df['Title'] = df['Title'].apply(lambda x: x.upper())

# Display the updated DataFrame
df.head()


,Title,UPC,Product_Type,Price_excl_tax,Price_incl_tax,Tax,Availability,Description,Reviews
0,A_LIGHT_IN_THE_ATTIC,a897fe39b1053632,books,â_51_77,â_51_77,â_0_00,in_stock__22_available_,_____it_s_hard_to_imagine_a_world_without_a_li...,0
1,TIPPING_THE_VELVET,90fa61229261140a,books,â_53_74,â_53_74,â_0_00,in_stock__20_available_,______erotic_and_absorbing___written_with_star...,0
2,SOUMISSION,6957f44c3847a760,books,â_50_10,â_50_10,â_0_00,in_stock__20_available_,_____dans_une_france_assez_proche_de_la_nã_tre...,0
3,SHARP_OBJECTS,e00eb4fd7b871a48,books,â_47_82,â_47_82,â_0_00,in_stock__20_available_,_____wicked_above_her_hipbone__girl_across_her...,0
4,SAPIENS__A_BRIEF_HISTORY_OF_HUMANKIND,4165285e1663650f,books,â_54_23,â_54_23,â_0_00,in_stock__20_available_,_____from_a_renowned_historian_comes_a_groundb...,0


In [6]:
# Function to clean and normalize the 'Description' column
def clean_description(description):
    return ' '.join(description.lower().split())

# Apply the function to the 'Description' column
df['Description'] = df['Description'].apply(clean_description)

# Display the updated DataFrame
df.head()


,Title,UPC,Product_Type,Price_excl_tax,Price_incl_tax,Tax,Availability,Description,Reviews
0,A_LIGHT_IN_THE_ATTIC,a897fe39b1053632,books,â_51_77,â_51_77,â_0_00,in_stock__22_available_,_____it_s_hard_to_imagine_a_world_without_a_li...,0
1,TIPPING_THE_VELVET,90fa61229261140a,books,â_53_74,â_53_74,â_0_00,in_stock__20_available_,______erotic_and_absorbing___written_with_star...,0
2,SOUMISSION,6957f44c3847a760,books,â_50_10,â_50_10,â_0_00,in_stock__20_available_,_____dans_une_france_assez_proche_de_la_nã_tre...,0
3,SHARP_OBJECTS,e00eb4fd7b871a48,books,â_47_82,â_47_82,â_0_00,in_stock__20_available_,_____wicked_above_her_hipbone__girl_across_her...,0
4,SAPIENS__A_BRIEF_HISTORY_OF_HUMANKIND,4165285e1663650f,books,â_54_23,â_54_23,â_0_00,in_stock__20_available_,_____from_a_renowned_historian_comes_a_groundb...,0


In [7]:
# Make 'Product Type' column values lowercase
df['Product_Type'] = df['Product_Type'].apply(lambda x: x.lower())

# Display the updated DataFrame
df.head()


,Title,UPC,Product_Type,Price_excl_tax,Price_incl_tax,Tax,Availability,Description,Reviews
0,A_LIGHT_IN_THE_ATTIC,a897fe39b1053632,books,â_51_77,â_51_77,â_0_00,in_stock__22_available_,_____it_s_hard_to_imagine_a_world_without_a_li...,0
1,TIPPING_THE_VELVET,90fa61229261140a,books,â_53_74,â_53_74,â_0_00,in_stock__20_available_,______erotic_and_absorbing___written_with_star...,0
2,SOUMISSION,6957f44c3847a760,books,â_50_10,â_50_10,â_0_00,in_stock__20_available_,_____dans_une_france_assez_proche_de_la_nã_tre...,0
3,SHARP_OBJECTS,e00eb4fd7b871a48,books,â_47_82,â_47_82,â_0_00,in_stock__20_available_,_____wicked_above_her_hipbone__girl_across_her...,0
4,SAPIENS__A_BRIEF_HISTORY_OF_HUMANKIND,4165285e1663650f,books,â_54_23,â_54_23,â_0_00,in_stock__20_available_,_____from_a_renowned_historian_comes_a_groundb...,0


In [9]:
# Function to extract numerical values from the specific format
def extract_numerical_value_special_format(text):
    # Remove non-alphanumeric characters except for '.' and digits
    cleaned_text = ''.join(c if c.isalnum() or c in ['.', '-'] else ' ' for c in text)
    
    # Split the cleaned text into words
    words = cleaned_text.split()
    
    # Try to convert each word to a float, return the first successful conversion
    for word in words:
        try:
            # If the word starts with 'â_', remove it
            if word.startswith('â_'):
                word = word[2:]
            
            # Convert to float and return
            return float(word)
        except ValueError:
            continue
    
    # Return None if no numerical value is found
    return None

# Apply the function to the specified columns
columns_to_convert = ['Price_excl_tax', 'Price_incl_tax', 'Tax', 'Availability']
for column in columns_to_convert:
    df[column] = df[column].apply(extract_numerical_value_special_format)

# Display the updated DataFrame
df.head()


,Title,UPC,Product_Type,Price_excl_tax,Price_incl_tax,Tax,Availability,Description,Reviews
0,A_LIGHT_IN_THE_ATTIC,a897fe39b1053632,books,51.0,51.0,0.0,22.0,_____it_s_hard_to_imagine_a_world_without_a_li...,0
1,TIPPING_THE_VELVET,90fa61229261140a,books,53.0,53.0,0.0,20.0,______erotic_and_absorbing___written_with_star...,0
2,SOUMISSION,6957f44c3847a760,books,50.0,50.0,0.0,20.0,_____dans_une_france_assez_proche_de_la_nã_tre...,0
3,SHARP_OBJECTS,e00eb4fd7b871a48,books,47.0,47.0,0.0,20.0,_____wicked_above_her_hipbone__girl_across_her...,0
4,SAPIENS__A_BRIEF_HISTORY_OF_HUMANKIND,4165285e1663650f,books,54.0,54.0,0.0,20.0,_____from_a_renowned_historian_comes_a_groundb...,0


In [10]:
import sqlite3

# Connect to SQLite database (this will create the database file if it doesn't exist)
conn = sqlite3.connect('books_database.db')

# Save the DataFrame to the database
df.to_sql('books_table', conn, index=False, if_exists='replace')

# Close the database connection
conn.close()
